In [1]:
# making some nice printing functions quick
import numpy as np
from pprint import pprint
np.set_printoptions(precision=4)

def print_dict(d):
    for k,v in d.items():
        print(k+':')
        pprint(v)
    print()

# Gaussian Elimination
I was suprised at how straightforward and simple it was to make an algorithm capable of solving large systems of 
linear equations. I had used functions to put an augmented matrix in reduced row echelon format 
in the past but never implemented my own Gaussian elimination. 

In [2]:
def gaussian_elimination(A,b):
    '''Takes a square matrix and returns it's inverse using gaussian elimination and back substitution'''
    a = np.hstack((A,b)) # using augmented matrix
    eps = 1e-16
    if a.shape[0]+1 != a.shape[1]:
        print("matrix must be augmented, instead had shape {}".format(a.shape))
        return
    n = a.shape[0] # num rows 
    # intializing permutor matrix
    P = np.eye(n)
    for c in range(0,n): # only go up to N because we only do elimination in square part

    #     #partial pivoting
        this_pivot = abs(a[c,c])
        maximum = this_pivot
        max_r = c
        for r in range(c+1,n):
            potential_pivot = abs(a[r,c])
            if potential_pivot>this_pivot:
                maximum = potential_pivot
                max_r = r
        if max_r!=c: # if we have a swap
            # construct row swap matrix
            rowswap = np.eye(n)
            rowswap[:,(max_r,c)] = rowswap[:,(c,max_r)] # swap corresponding columns to get ith permutor
            P = rowswap@P # record the fact that the columns were swapped
            a = rowswap@a # go home, turn off your spigot, everybody do the partial pivot

        for r in range(c+1,n):
            piv = a[c,c] # pivot point
            entry = a[r,c] # the value we are trying to eliminate
            a[r,:] -= a[c,:]*(entry/piv) # eliminating `entry`

    U = a.copy()[:,:-1] # Make upper triangular matrix from a at this at point in time,
    c = a.copy()[:,-1].reshape((-1,1)) # as well as the c vector (just the last column of the augmented matrix)

    x = np.empty((n,1)) # make an empty column vector of the appropriate size
    # Doing back substitution
    for i in range(n-1,-1,-1):
        x[i] = (a[i,-1]-a[i,i+1:-1]@x[i+1:,0])/a[i,i] 
        # sub = a[i,i+1:-1]@x[i+1:,0]
        # divide the value in the c minus the already solved vars in column vector 
        # matrix multiplied by the row vector formed by the non pivot entries to the right of the pivot in each column
        # divided by the pivot value in the same row 

    # Note that the rows in X have been swapped if a partial pivot has occurred, undo the swaps so we can return X with the variables
    # in the correct columns
    return {'x':x,'U':U,'c':c,'P':P}

    # Doing Gaussian Elimination on lower triangular matrix
    # for c in range(n-1,-1,-1): 
    #     a[c,:] /= a[c,c] # making pivot point 1 and entry in augmented portion correct
    #     for r in range(c-1,-1,-1):
    #         a[r,:] -= a[c,:]*a[r,c] # eliminating each entry in the upper portion of the matrix

In [3]:
# Example 6
A = np.array([[1,2,1],\
               [3,8,1],\
               [0,4,1]])
b = np.array([2,12,2]).reshape((3,1))
print_dict(gaussian_elimination(A,b))

x:
array([[ 2.],
       [ 1.],
       [-2.]])
U:
array([[3.    , 8.    , 1.    ],
       [0.    , 4.    , 1.    ],
       [0.    , 0.    , 0.8333]])
c:
array([[12.    ],
       [ 2.    ],
       [-1.6667]])
P:
array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])



In [4]:
# Example 12
A = np.array([[2,6,10],\
               [1,3,3],\
               [3,14,28]])
b = np.array([0,2,-8]).reshape((3,1))
print_dict(gaussian_elimination(A,b)) # pivot on the haters

x:
array([[ 2.],
       [ 1.],
       [-1.]])
U:
array([[ 3.    , 14.    , 28.    ],
       [ 0.    , -3.3333, -8.6667],
       [ 0.    ,  0.    , -2.    ]])
c:
array([[-8.    ],
       [ 5.3333],
       [ 2.    ]])
P:
array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])



# LU Decomposition
This one posed more of a challenge to me. I realized that $L^{-1}$ was built out of all of 
the ERO matricies cumulatively left multiplying A one after another. This would mean that
similarly $L$ was all of the *inverses* of the ERO's cumulatively *right* multiplied to an 
identity matrix.

In [5]:


def my_lu(a):
    '''Takes a square matrix and returns it's L U decomposition'''
    eps = 1e-16
    if a.shape[0] != a.shape[1]:
        print("matrix must be square, instead had shape {}".format(a.shape))
        return
    n = a.shape[0] # num rows 
    # intializing permutor matrix
    I = np.eye(n)
    P = np.eye(n)
    L = np.eye(n)
    for c in range(0,n): # only go up to N because we only do elimination in square part
    #     #partial pivoting
        this_pivot = abs(a[c,c])
        maximum = this_pivot
        max_r = c
        for r in range(c+1,n):
            potential_pivot = abs(a[r,c])
            if potential_pivot>this_pivot:
                maximum = potential_pivot
                max_r = r
        if max_r!=c: # if we have a swap
            # construct row swap matrix
            rowswap = np.eye(n)
            rowswap[:,(max_r,c)] = rowswap[:,(c,max_r)] # swap corresponding columns to get ith permutor
            L = L@rowswap.T # record the fact that the columns were swapped in the L matrix
            a = rowswap@a # go home, turn off your spigot, everybody do the partial pivot

        for r in range(c+1,n):
            piv = a[c,c] # pivot point
            entry = a[r,c] # the value we are trying to eliminate
            ERO = I.copy()
            ERO_inv = I.copy()
            ERO[r,c] = -(entry/piv)
            ERO_inv[r,c] = (entry/piv) # "Undoes" the row operation represented by ERO
            a = ERO@a
            L = L@ERO_inv

    return {'L':L,'U':a}

In [6]:
A = np.array([[1,2,1],\
               [3,8,1],\
               [0,4,1]])
d = my_lu(A)
L = d["L"]
U = d["U"]
print_dict(d) # note that L is not lower triangular, but permuted lower triangular
print("Recomposed:")
print(L@U)

L:
array([[ 0.3333, -0.1667,  1.    ],
       [ 1.    ,  0.    ,  0.    ],
       [ 0.    ,  1.    ,  0.    ]])
U:
array([[3.    , 8.    , 1.    ],
       [0.    , 4.    , 1.    ],
       [0.    , 0.    , 0.8333]])

Recomposed:
[[1. 2. 1.]
 [3. 8. 1.]
 [0. 4. 1.]]


In [7]:
A = np.array([[2,6,10],\
               [1,3,3],\
               [3,14,28]])
d = my_lu(A)
L = d["L"]
U = d["U"]
print_dict(d)
print("Recomposed:")
print(L@U)

L:
array([[0.6667, 1.    , 0.    ],
       [0.3333, 0.5   , 1.    ],
       [1.    , 0.    , 0.    ]])
U:
array([[ 3.    , 14.    , 28.    ],
       [ 0.    , -3.3333, -8.6667],
       [ 0.    ,  0.    , -2.    ]])

Recomposed:
[[ 2.  6. 10.]
 [ 1.  3.  3.]
 [ 3. 14. 28.]]


# Testing NumPy Orthogonalization

In [8]:

# Yanked from stackoverflow :)
# https://stackoverflow.com/questions/47834140/numpy-equivalent-of-matlabs-magic
def magic(n):
  n = int(n)
  if n < 3:
    raise ValueError("Size must be at least 3")
  if n % 2 == 1:
    p = np.arange(1, n+1)
    return (n*np.mod(p[:, None] + p - (n+3)//2, n) + np.mod(p[:, None] + 2*p-2, n) + 1).astype(float)
  elif n % 4 == 0:
    J = np.mod(np.arange(1, n+1), 4) // 2
    K = J[:, None] == J
    M = np.arange(1, n*n+1, n)[:, None] + np.arange(n)
    M[K] = n*n + 1 - M[K]
  else:
    p = n//2
    M = magic(p)
    M = np.block([[M, M+2*p*p], [M+3*p*p, M+p*p]])
    i = np.arange(p)
    k = (n-2)//4
    j = np.concatenate((np.arange(k), np.arange(n-k+1, n)))
    M[np.ix_(np.concatenate((i, i+p)), j)] = M[np.ix_(np.concatenate((i+p, i)), j)]
    M[np.ix_([k, k+p], [0, k])] = M[np.ix_([k+p, k], [0, k])]
  return M.astype(float) # changing the output of this to be of type float to make visual comparisons easier

mag = magic(5)
mag

array([[17., 24.,  1.,  8., 15.],
       [23.,  5.,  7., 14., 16.],
       [ 4.,  6., 13., 20., 22.],
       [10., 12., 19., 21.,  3.],
       [11., 18., 25.,  2.,  9.]])

In [9]:
# Comparing Q*R for Q and R produced by Numpy function for QR decomposition to original matrix
# for magic matricies with 3,5 and 10 entries
for size in [3,5,10]:
    m = magic(size)
    q,r = np.linalg.qr(m)
    print("Original Matrix:\n{}".format(m))
    print("Recomposed from QR decomposition:\n{}".format(q@r))

Original Matrix:
[[8. 1. 6.]
 [3. 5. 7.]
 [4. 9. 2.]]
Recomposed from QR decomposition:
[[8. 1. 6.]
 [3. 5. 7.]
 [4. 9. 2.]]
Original Matrix:
[[17. 24.  1.  8. 15.]
 [23.  5.  7. 14. 16.]
 [ 4.  6. 13. 20. 22.]
 [10. 12. 19. 21.  3.]
 [11. 18. 25.  2.  9.]]
Recomposed from QR decomposition:
[[17. 24.  1.  8. 15.]
 [23.  5.  7. 14. 16.]
 [ 4.  6. 13. 20. 22.]
 [10. 12. 19. 21.  3.]
 [11. 18. 25.  2.  9.]]
Original Matrix:
[[ 92.  99.   1.   8.  15.  67.  74.  51.  58.  40.]
 [ 98.  80.   7.  14.  16.  73.  55.  57.  64.  41.]
 [  4.  81.  88.  20.  22.  54.  56.  63.  70.  47.]
 [ 85.  87.  19.  21.   3.  60.  62.  69.  71.  28.]
 [ 86.  93.  25.   2.   9.  61.  68.  75.  52.  34.]
 [ 17.  24.  76.  83.  90.  42.  49.  26.  33.  65.]
 [ 23.   5.  82.  89.  91.  48.  30.  32.  39.  66.]
 [ 79.   6.  13.  95.  97.  29.  31.  38.  45.  72.]
 [ 10.  12.  94.  96.  78.  35.  37.  44.  46.  53.]
 [ 11.  18. 100.  77.  84.  36.  43.  50.  27.  59.]]
Recomposed from QR decomposition:
[[ 92.  99

The two are equivalent.
Showing that column vectors of Q are orthogonal:

In [10]:
q,r = np.linalg.qr(mag)
from itertools import combinations
for a,b in combinations(list(range(5)),2):
    print("Column {}: {}".format(a,q[:,a]))
    print("Column {}: {}".format(b,q[:,b]))
    print("Column {} dot Column {}: {}\n".format(a,b,q[:,a]@q[:,b]))

Column 0: [-0.5234 -0.7081 -0.1231 -0.3079 -0.3387]
Column 1: [ 0.5058 -0.6966  0.1367  0.1911  0.4514]
Column 0 dot Column 1: -5.551115123125783e-17

Column 0: [-0.5234 -0.7081 -0.1231 -0.3079 -0.3387]
Column 2: [ 0.6735 -0.0177 -0.3558 -0.4122 -0.4996]
Column 0 dot Column 2: 0.0

Column 0: [-0.5234 -0.7081 -0.1231 -0.3079 -0.3387]
Column 3: [-0.1215  0.0815 -0.6307 -0.4247  0.6328]
Column 0 dot Column 3: -2.7755575615628914e-17

Column 0: [-0.5234 -0.7081 -0.1231 -0.3079 -0.3387]
Column 4: [-0.0441 -0.08   -0.6646  0.72   -0.1774]
Column 0 dot Column 4: 1.3877787807814457e-17

Column 1: [ 0.5058 -0.6966  0.1367  0.1911  0.4514]
Column 2: [ 0.6735 -0.0177 -0.3558 -0.4122 -0.4996]
Column 1 dot Column 2: 5.551115123125783e-17

Column 1: [ 0.5058 -0.6966  0.1367  0.1911  0.4514]
Column 3: [-0.1215  0.0815 -0.6307 -0.4247  0.6328]
Column 1 dot Column 3: 5.551115123125783e-17

Column 1: [ 0.5058 -0.6966  0.1367  0.1911  0.4514]
Column 4: [-0.0441 -0.08   -0.6646  0.72   -0.1774]
Column 1 d

# Testing NumPy Eigenvalue Solving

In [11]:
eig_vals,eig_vecs = np.linalg.eig(mag)
pprint(eig_vals)
pprint(eig_vecs)

array([ 65.    , -21.2768, -13.1263,  21.2768,  13.1263])
array([[-0.4472,  0.0976, -0.633 ,  0.678 , -0.2619],
       [-0.4472,  0.3525,  0.5895,  0.3223, -0.1732],
       [-0.4472,  0.5501, -0.3915, -0.5501,  0.3915],
       [-0.4472, -0.3223,  0.1732, -0.3525, -0.5895],
       [-0.4472, -0.678 ,  0.2619, -0.0976,  0.633 ]])


Comparing $ A x $ and $ \lambda x $ 

In [12]:
for i in range(mag.shape[0]):
    print("{}/ Value of |A*v|: {}".format(i,mag@eig_vecs[:,i]))
    print("{}/ Value of λ:     {}".format(i,eig_vals[i]*eig_vecs[:,i]))

0/ Value of |A*v|: [-29.0689 -29.0689 -29.0689 -29.0689 -29.0689]
0/ Value of λ:     [-29.0689 -29.0689 -29.0689 -29.0689 -29.0689]
1/ Value of |A*v|: [ -2.0775  -7.5009 -11.7046   6.8571  14.4259]
1/ Value of λ:     [ -2.0775  -7.5009 -11.7046   6.8571  14.4259]
2/ Value of |A*v|: [ 8.3086 -7.7377  5.1393 -2.273  -3.4373]
2/ Value of λ:     [ 8.3086 -7.7377  5.1393 -2.273  -3.4373]
3/ Value of |A*v|: [ 14.4259   6.8571 -11.7046  -7.5009  -2.0775]
3/ Value of λ:     [ 14.4259   6.8571 -11.7046  -7.5009  -2.0775]
4/ Value of |A*v|: [-3.4373 -2.273   5.1393 -7.7377  8.3086]
4/ Value of λ:     [-3.4373 -2.273   5.1393 -7.7377  8.3086]
